In [34]:
from numpy import expand_dims
from numpy import zeros
from numpy import ones
from numpy import vstack
from numpy.random import randn
from numpy.random import randint
from keras.datasets.mnist import load_data
from tensorflow.keras.optimizers import Adam
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Flatten
from keras.layers import Conv2D
from keras.layers import Conv2DTranspose
from keras.layers import LeakyReLU
from keras.layers import Dropout
from matplotlib import pyplot
import pandas as pd
import cv2
import numpy as np
from imblearn.over_sampling import RandomOverSampler
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.models import load_model

In [35]:
data = pd.read_csv('/content/drive/MyDrive/fer2013.csv')
data.head()

,emotion,pixels,Usage
0,0,70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...,Training
1,0,151 150 147 155 148 133 111 140 170 174 182 15...,Training
2,2,231 212 156 164 174 138 161 173 182 200 106 38...,Training
3,4,24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...,Training
4,6,4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...,Training


In [36]:
pixel_data = data['pixels']
label_data = data['emotion']

In [6]:
def preprocess_pixels(pixel_data):
  images = []
  for i in range(len(pixel_data)):
    img = np.fromstring(pixel_data[i], dtype=np.uint8, sep=' ')
    img = cv2.resize(img, (28, 28))
    img = cv2.fastNlMeansDenoising(img,None,3,7,21)
    images.append(img)

  X = np.array(images)

 

  return X

In [7]:
oversampler = RandomOverSampler(sampling_strategy='auto')

X_over, Y_over = oversampler.fit_resample(pixel_data.values.reshape(-1,1), label_data)

X_over_series = pd.Series(X_over.flatten())

In [8]:
X = preprocess_pixels(X_over_series)
Y = Y_over
Y = Y.values.reshape(-1,1)

In [37]:
print("Images shape = ",X.shape,"\nLabels shape = ",Y.shape)
print(type(X),type(Y))

Images shape =  (62923, 28, 28) 
Labels shape =  (62923, 1)
<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [10]:
#1-step in data shuffling

#get equally spaced numbers in a given range
n = np.arange(X.shape[0])
print("'n' values before shuffling = ",n)

#shuffle all the equally spaced values in list 'n'
np.random.seed(45)
np.random.shuffle(n)
print("\n'n' values after shuffling = ",n)

'n' values before shuffling =  [    0     1     2 ... 62920 62921 62922]

'n' values after shuffling =  [62634  6425 11709 ...  6012  6558 56267]


In [11]:
#2-step in data shuffling

#shuffle images and corresponding labels data in both the lists
X = X[n]
Y = Y[n]

print("Images shape after shuffling = ",X.shape,"\nLabels shape after shuffling = ",Y.shape)

Images shape after shuffling =  (62923, 28, 28) 
Labels shape after shuffling =  (62923, 1)


In [12]:
trainX, _, _, _ = train_test_split(X, Y, test_size = 0.2, random_state = 45)
print("x_train shape = ",trainX.shape)

x_train shape =  (50338, 28, 28)


In [13]:
# define the standalone discriminator model
def define_discriminator(in_shape=(28,28,1)):
	model = Sequential()
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same', input_shape=in_shape))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Conv2D(64, (3,3), strides=(2, 2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dropout(0.4))
	model.add(Flatten())
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])
	return model

In [14]:
# define the standalone generator model
def define_generator(latent_dim):
	model = Sequential()
	# foundation for 7x7 image
	n_nodes = 128 * 7 * 7
	model.add(Dense(n_nodes, input_dim=latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Reshape((7, 7, 128)))
	# upsample to 14x14
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# upsample to 28x28
	model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv2D(1, (7,7), activation='softmax', padding='same'))
	return model

In [15]:
# define the combined generator and discriminator model, for updating the generator
def define_gan(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	# add the discriminator
	model.add(d_model)
	# compile model
	opt = Adam(lr=0.0002, beta_1=0.5)
	model.compile(loss='binary_crossentropy', optimizer=opt)
	return model

In [16]:
# load and prepare fer training images
def load_real_samples():
  # expand to 3d, e.g. add channels dimension
	X = expand_dims(trainX, axis=-1)
	# convert from unsigned ints to floats
	X = X.astype('float32')
	# scale from [0,255] to [0,1]
	X = X / 255.0
	return X

In [17]:
# select real samples
def generate_real_samples(dataset, n_samples):
	# choose random instances
	ix = randint(0, dataset.shape[0], n_samples)
	# retrieve selected images
	X = dataset[ix]
	# generate 'real' class labels (1)
	y = ones((n_samples, 1))
	return X, y

In [18]:
# generate points in latent space as input for the generator
def generate_latent_points(latent_dim, n_samples):
	# generate points in the latent space
	x_input = randn(latent_dim * n_samples)
	# reshape into a batch of inputs for the network
	x_input = x_input.reshape(n_samples, latent_dim)
	return x_input

In [19]:
# use the generator to generate n fake examples, with class labels
def generate_fake_samples(g_model, latent_dim, n_samples):
	# generate points in latent space
	x_input = generate_latent_points(latent_dim, n_samples)
	# predict outputs
	X = g_model.predict(x_input)
	# create 'fake' class labels (0)
	y = zeros((n_samples, 1))
	return X, y

In [20]:
# create and save a plot of generated images (reversed grayscale)
def save_plot(examples, epoch, n=10):
	# plot images
	for i in range(n * n):
		# define subplot
		pyplot.subplot(n, n, 1 + i)
		# turn off axis
		pyplot.axis('off')
		# plot raw pixel data
		pyplot.imshow(examples[i, :, :, 0], cmap='gray_r')
	# save plot to file
	filename = 'generated_plot_e%03d.png' % (epoch+1)
	pyplot.savefig(filename)
	pyplot.close()

In [21]:
# evaluate the discriminator, plot generated images, save generator model
def summarize_performance(epoch, g_model, d_model, dataset, latent_dim, n_samples=100):
	# prepare real samples
	X_real, y_real = generate_real_samples(dataset, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(X_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_samples(g_model, latent_dim, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	save_plot(x_fake, epoch)
	# save the generator model tile file
	filename = 'generator_model_%03d.h5' % (epoch + 1)
	g_model.save(filename)

In [22]:
# train the generator and discriminator
def train(g_model, d_model, gan_model, dataset, latent_dim, n_epochs=15, n_batch=256):
	bat_per_epo = int(dataset.shape[0] / n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(bat_per_epo):
			# get randomly selected 'real' samples
			X_real, y_real = generate_real_samples(dataset, half_batch)
			# generate 'fake' examples
			X_fake, y_fake = generate_fake_samples(g_model, latent_dim, half_batch)
			# create training set for the discriminator
			X, y = vstack((X_real, X_fake)), vstack((y_real, y_fake))
			# update discriminator model weights
			d_loss, _ = d_model.train_on_batch(X, y)
			# prepare points in latent space as input for the generator
			X_gan = generate_latent_points(latent_dim, n_batch)
			# create inverted labels for the fake samples
			y_gan = ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(X_gan, y_gan)
			# summarize loss on this batch
			print('>%d, %d/%d, d=%.3f, g=%.3f' % (i+1, j+1, bat_per_epo, d_loss, g_loss))
		# evaluate the model performance, sometimes
		if (i+1) % 10 == 0:
			summarize_performance(i, g_model, d_model, dataset, latent_dim)

In [23]:
# size of the latent space
latent_dim = 100
# create the discriminator
d_model = define_discriminator()
# create the generator
g_model = define_generator(latent_dim)
# create the gan
gan_model = define_gan(g_model, d_model)
# load image data
dataset = load_real_samples()
# train model
train(g_model, d_model, gan_model, dataset, latent_dim)

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


>1, 1/196, d=0.689, g=0.773
>1, 2/196, d=0.679, g=0.813
>1, 3/196, d=0.673, g=0.863
>1, 4/196, d=0.669, g=0.902
>1, 5/196, d=0.659, g=0.950
>1, 6/196, d=0.654, g=0.989
>1, 7/196, d=0.654, g=1.014
>1, 8/196, d=0.647, g=1.036
>1, 9/196, d=0.647, g=1.073
>1, 10/196, d=0.642, g=1.102
>1, 11/196, d=0.634, g=1.125
>1, 12/196, d=0.642, g=1.131
>1, 13/196, d=0.639, g=1.150
>1, 14/196, d=0.632, g=1.166
>1, 15/196, d=0.636, g=1.169
>1, 16/196, d=0.630, g=1.161
>1, 17/196, d=0.632, g=1.173
>1, 18/196, d=0.623, g=1.182
>1, 19/196, d=0.621, g=1.195
>1, 20/196, d=0.621, g=1.202
>1, 21/196, d=0.616, g=1.202
>1, 22/196, d=0.622, g=1.194
>1, 23/196, d=0.615, g=1.187
>1, 24/196, d=0.616, g=1.195
>1, 25/196, d=0.604, g=1.190
>1, 26/196, d=0.603, g=1.196
>1, 27/196, d=0.613, g=1.206
>1, 28/196, d=0.592, g=1.207
>1, 29/196, d=0.591, g=1.218
>1, 30/196, d=0.589, g=1.244
>1, 31/196, d=0.592, g=1.237
>1, 32/196, d=0.577, g=1.277
>1, 33/196, d=0.573, g=1.270
>1, 34/196, d=0.569, g=1.271
>1, 35/196, d=0.563, g=

>11, 1/196, d=0.000, g=7.656
>11, 2/196, d=0.000, g=7.727
>11, 3/196, d=0.000, g=7.815
>11, 4/196, d=0.000, g=8.109
>11, 5/196, d=0.000, g=8.177
>11, 6/196, d=0.002, g=7.776
>11, 7/196, d=0.000, g=7.724
>11, 8/196, d=0.024, g=3.700
>11, 9/196, d=0.016, g=10.639
>11, 10/196, d=0.000, g=13.162
>11, 11/196, d=0.007, g=13.409
>11, 12/196, d=0.009, g=12.141
>11, 13/196, d=0.000, g=11.389
>11, 14/196, d=0.000, g=11.034
>11, 15/196, d=0.017, g=9.591
>11, 16/196, d=0.000, g=8.894
>11, 17/196, d=0.000, g=8.606
>11, 18/196, d=0.000, g=8.384
>11, 19/196, d=0.000, g=8.294
>11, 20/196, d=0.000, g=8.407
>11, 21/196, d=0.011, g=7.358
>11, 22/196, d=0.000, g=6.857
>11, 23/196, d=0.001, g=6.911
>11, 24/196, d=0.001, g=7.002
>11, 25/196, d=0.001, g=7.252
>11, 26/196, d=0.002, g=6.960
>11, 27/196, d=0.001, g=7.116
>11, 28/196, d=0.001, g=7.298
>11, 29/196, d=0.000, g=7.403
>11, 30/196, d=0.000, g=7.654
>11, 31/196, d=0.001, g=7.702
>11, 32/196, d=0.000, g=7.808
>11, 33/196, d=0.001, g=7.775
>11, 34/196, 

In [33]:
#generator_model = load_model('generator_model_010.h5')
X_real, y_real = generate_real_samples(dataset, 128)

_, accuracy = d_model.evaluate(X_real, y_real, verbose=0)  
print('test_accuracy: %.2f' % (accuracy * 100.0))

X_fake, y_fake = generate_fake_samples(g_model, latent_dim, 128)
_, accuracy = d_model.evaluate(X_fake, y_fake, verbose=0)
print('test_accuracy: %.2f' % (accuracy * 100.0))

test_accuracy: 100.00
test_accuracy: 100.00
